# Day 7
https://adventofcode.com/2016/day/7

In [1]:
import aocd
data = aocd.get_data(year=2016, day=7)

In [2]:
from dataclasses import dataclass
import re

In [3]:
re_hypernet = re.compile(r'(\[\w+\])')
re_abba = re.compile(r'(\w)((?!\1)\w)(\2)(\1)')
re_aba = re.compile(r'(\w)((?!\1)\w)(\1)')

In [4]:
def findall(regex, text):
    """Emulate regex.findall(text), but with support for overlapping groups."""
    reported = set()
    pos = 0
    while pos < len(text):
        match = regex.search(text, pos)
        if match:
            if match.start() not in reported:
                yield match.groups()
                reported.add(match.start())
        pos += 1

In [5]:
def reverse_aba(aba):
    return aba[1] + aba[0] + aba[1]

In [6]:
@dataclass(frozen=True)
class Hostname():
    text: str
    
    @property
    def hypernet(self):
        return [substr for substr in re_hypernet.split(self.text) if substr[0] == '[']
    
    @property
    def supernet(self):
        return [substr for substr in re_hypernet.split(self.text) if substr[0] != '[']
    
    @property
    def supports_tls(self):
        hypernet_abbas = sum(1 for phrase in self.hypernet if re_abba.search(phrase) is not None)
        supernet_abbas = sum(1 for phrase in self.supernet if re_abba.search(phrase) is not None)
        return hypernet_abbas == 0 and supernet_abbas > 0
    
    @property
    def supports_ssl(self):
        hypernet_abas = [''.join(aba) for aba in findall(re_aba, '|'.join(self.hypernet))]
        supernet_abas = [''.join(aba) for aba in findall(re_aba, '|'.join(self.supernet))]
        return sum(1 for aba in supernet_abas if reverse_aba(aba) in hypernet_abas) > 0

In [7]:
hostnames = [Hostname(line) for line in data.split('\n')]
p1 = sum(1 for host in hostnames if host.supports_tls)
print('Part 1: {}'.format(p1))
p2 = sum(1 for host in hostnames if host.supports_ssl)
print('Part 2: {}'.format(p2))

Part 1: 105
Part 2: 258
